In [1]:
import sys
print('Python Version %s' % sys.version)

Python Version 3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]


In [2]:
!pip list

Package                            Version   
---------------------------------- ----------
alabaster                          0.7.10    
anaconda-client                    1.6.14    
anaconda-project                   0.8.2     
asn1crypto                         0.24.0    
astroid                            1.6.3     
astropy                            3.0.2     
attrs                              18.1.0    
Automat                            0.3.0     
autovizwidget                      0.15.0    
awscli                             1.18.39   
Babel                              2.5.3     
backcall                           0.1.0     
backports.shutil-get-terminal-size 1.0.0     
bcrypt                             3.1.7     
beautifulsoup4                     4.6.0     
bitarray                           0.8.1     
bkcharts                           0.2       
blaze                              0.11.3    
bleach                             2.1.3     
bokeh                             

In [3]:
%%bash
conda list

# packages in environment at /home/ec2-user/anaconda3/envs/python3:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
alabaster                 0.7.10           py36h306e16b_0  
anaconda-client           1.6.14                   py36_0  
anaconda-project          0.8.2            py36h44fb852_0  
argparse                  1.4.0                     <pip>
asn1crypto                0.24.0                   py36_0  
astroid                   1.6.3                    py36_0  
astropy                   3.0.2            py36h3010b51_1  
attrs                     18.1.0                   py36_0  
Automat                   0.3.0                     <pip>
autovizwidget             0.15.0                    <pip>
awscli                    1.18.39                   <pip>
babel                     2.5.3                    py36_0  
backcall                  0.1.0                    py36_0  
backports                 1.0  

In [4]:
%%html

<iframe src="https://datascienceonaws.com" width="800px" height="600px"/>

# 1. Install the CLI tools

_Note:  Make sure the SageMaker role has access to create EKS Cluster._

In [5]:
%%bash

source ~/.bash_profile

#### Install `eksctl`
# To get started we'll first install the `awscli` and `eksctl` CLI tools. [eksctl](https://eksctl.io) simplifies the process of creating EKS clusters.

pip install awscli --upgrade --user

curl --silent --location "https://github.com/weaveworks/eksctl/releases/latest/download/eksctl_$(uname -s)_amd64.tar.gz" | tar xz -C /tmp

sudo mv /tmp/eksctl /usr/local/bin

eksctl version

#### Install `kubectl`
# `kubectl` is a command line interface for running commands against Kubernetes clusters. 
# Run the following to install Kubectl

curl --location -o ./kubectl https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/kubectl

chmod +x ./kubectl

sudo mv ./kubectl /usr/local/bin

kubectl version --short --client

#### Install `aws-iam-authenticator`

curl -o aws-iam-authenticator https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/aws-iam-authenticator

chmod +x ./aws-iam-authenticator

sudo mv aws-iam-authenticator /usr/local/bin

aws-iam-authenticator version

#### Install jq and envsubst (from GNU gettext utilities) 
sudo yum -y install jq gettext

#### Verify the binaries are in the path and executable
for command in kubectl jq envsubst
  do
    which $command &>/dev/null && echo "$command in path" || echo "$command NOT FOUND"
  done

echo "Completed"

0.17.0
Client Version: v1.15.10-eks-bac369
{"Version":"v0.5.0","Commit":"1cfe2a90f68381eacd7b6dcfa2bf689e76eb8b4b"}
Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
Package jq-1.5-1.2.amzn1.x86_64 already installed and latest version
Package gettext-0.18.1.1-9.1.11.amzn1.x86_64 already installed and latest version
Nothing to do
kubectl in path
jq in path
envsubst in path
Completed


You should consider upgrading via the 'pip install --upgrade pip' command.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 55.1M  100 55.1M    0     0  65.4M      0 --:--:-- --:--:-- --:--:-- 65.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.6M  100 33.6M    0     0  86.7M      0 --:--:-- --:--:-- --:--:-- 86.7M


# 2. Configure the Environment Variables

In [6]:
%%bash

source ~/.bash_profile

export AWS_REGION=$(aws configure get region)
echo "export AWS_REGION=${AWS_REGION}" | tee -a ~/.bash_profile

export AWS_CLUSTER_NAME=cluster
echo "export AWS_CLUSTER_NAME=${AWS_CLUSTER_NAME}" | tee -a ~/.bash_profile

echo "Completed"

export AWS_REGION=us-west-2
export AWS_CLUSTER_NAME=cluster
Completed


# 3. Create the EKS Cluster

_Note:  Creating a cluster may take about 15 minutes.  Please be patient._

In [7]:
%%bash

source ~/.bash_profile

cat << EOF > cluster.yaml
---
apiVersion: eksctl.io/v1alpha5
kind: ClusterConfig

metadata:
  name: ${AWS_CLUSTER_NAME}
  region: ${AWS_REGION}

cloudWatch:
  clusterLogging:
    enableTypes: ["*"]

managedNodeGroups:
- name: cpu-nodes
  instanceType: c5.xlarge
  volumeSize: 100
  desiredCapacity: 5
  iam:
    withAddonPolicies:
      albIngress: true

#secretsEncryption:
#  keyARN: ${MASTER_ARN}
EOF

In [8]:
!cat cluster.yaml

---
apiVersion: eksctl.io/v1alpha5
kind: ClusterConfig

metadata:
  name: cluster
  region: us-west-2

cloudWatch:
  clusterLogging:
    enableTypes: ["*"]

managedNodeGroups:
- name: cpu-nodes
  instanceType: c5.xlarge
  volumeSize: 100
  desiredCapacity: 5
  iam:
    withAddonPolicies:
      albIngress: true

#secretsEncryption:
#  keyARN: 


In [ ]:
%%bash

source ~/.bash_profile

eksctl create cluster -f ./cluster.yaml

echo "Completed"

In [ ]:
%%bash

source ~/.bash_profile

eksctl utils describe-stacks --region=${AWS_REGION} --cluster=${AWS_CLUSTER_NAME}

# --- Please wait until the EKS cluster is succesfully deployed before you continue! ---

![](img/verify_eks.png)

# 4. Associate IAM Policies with EKS Worker Nodes


In [ ]:
%%bash

### Source the environment
source ~/.bash_profile

### Create more environment variables
export STACK_NAME=$(eksctl get nodegroup --cluster ${AWS_CLUSTER_NAME} -o json | jq -r '.[].StackName')
echo "export STACK_NAME=${STACK_NAME}" | tee -a ~/.bash_profile

export INSTANCE_ROLE_NAME=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .RoleName")
#export INSTANCE_ROLE_NAME=$(aws cloudformation describe-stacks --stack-name $STACK_NAME --output text --query "Stacks[0].Outputs[1].OutputValue" | sed -e 's/.*\///g')
echo "export INSTANCE_ROLE_NAME=${INSTANCE_ROLE_NAME}" | tee -a ~/.bash_profile

#export INSTANCE_PROFILE_ARN=$(aws cloudformation describe-stacks --stack-name $STACK_NAME | jq -r '.Stacks[].Outputs[] | select(.OutputKey=="InstanceProfileARN") | .OutputValue')
export INSTANCE_PROFILE_ARN=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .Arn")
echo "export INSTANCE_PROFILE_ARN=${INSTANCE_PROFILE_ARN}" | tee -a ~/.bash_profile

#### Allow Access from/to the Elastic Container Registry (ECR)
# This allows our cluster worker nodes to load custom Docker images (ie. models) from ECR.  We will load these custom Docker images in a later section.
aws iam attach-role-policy --role-name $INSTANCE_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

### Associated IAM and OIDC
# To use IAM roles for service accounts in your cluster, you must create an OIDC identity provider in the IAM console.  See https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html for more info.
eksctl utils associate-iam-oidc-provider --cluster ${AWS_CLUSTER_NAME} --approve
aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.identity.oidc.issuer" --output text

echo "Completed"

# 5. Update `~/.kube/config` with our new EKS cluster

In [ ]:
%%bash

source ~/.bash_profile

aws eks --region ${AWS_REGION} update-kubeconfig --name ${AWS_CLUSTER_NAME} 

# 6. Verify Your EKS Cluster


In [ ]:
%%bash

kubectl get ns

### EXPECTED OUTPUT ###
# NAME              STATUS   AGE
# default           Active   12m
# kube-node-lease   Active   13m
# kube-public       Active   13m
# kube-system       Active   13m


In [ ]:
%%bash

kubectl get nodes

### EXPECTED OUTPUT ###
# NAME                                            STATUS   ROLES    AGE    VERSION
# ip-XXX-XXX-XXX-XXX.us-west-2.compute.internal   Ready    <none>   7m3s   v1.14.7-eks-1861c5
# ip-XXX-XXX-XXX-XXX.us-west-2.compute.internal   Ready    <none>   7m4s   v1.14.7-eks-1861c5